Lambda School Data Science

*Unit 2, Sprint 1, Module 4*

---

# Logistic Regression


## Assignment 🌯

You'll use a [**dataset of 400+ burrito reviews**](https://srcole.github.io/100burritos/). How accurately can you predict whether a burrito is rated 'Great'?

> We have developed a 10-dimensional system for rating the burritos in San Diego. ... Generate models for what makes a burrito great and investigate correlations in its dimensions.

- [ ] Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.
- [ ] Begin with baselines for classification.
- [ ] Use scikit-learn for logistic regression.
- [ ] Get your model's validation accuracy. (Multiple times if you try multiple iterations.)
- [ ] Get your model's test accuracy. (One time, at the end.)
- [ ] Commit your notebook to your fork of the GitHub repo.


## Stretch Goals

- [ ] Add your own stretch goal(s) !
- [ ] Make exploratory visualizations.
- [ ] Do one-hot encoding.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Get and plot your coefficients.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

#Inital reading/ cleaning 

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [2]:
# Load data downloaded from https://srcole.github.io/100burritos/
import pandas as pd
df = pd.read_csv(DATA_PATH+'burritos/burritos.csv')

In [3]:
# Derive binary classification target:
# We define a 'Great' burrito as having an
# overall rating of 4 or higher, on a 5 point scale.
# Drop unrated burritos.
df = df.dropna(subset=['overall'])
df['Great'] = df['overall'] >= 4

In [4]:
# Clean/combine the Burrito categories
df['Burrito'] = df['Burrito'].str.lower()

california = df['Burrito'].str.contains('california')
asada = df['Burrito'].str.contains('asada')
surf = df['Burrito'].str.contains('surf')
carnitas = df['Burrito'].str.contains('carnitas')

df.loc[california, 'Burrito'] = 'California'
df.loc[asada, 'Burrito'] = 'Asada'
df.loc[surf, 'Burrito'] = 'Surf & Turf'
df.loc[carnitas, 'Burrito'] = 'Carnitas'
df.loc[~california & ~asada & ~surf & ~carnitas, 'Burrito'] = 'Other'

In [5]:
# Drop some high cardinality categoricals
df = df.drop(columns=['Notes', 'Location', 'Reviewer', 'Address', 'URL', 'Neighborhood'])

In [6]:
# Drop some columns to prevent "leakage"
df = df.drop(columns=['Rec', 'overall'])

In [7]:
#looking into the data
print(df.shape)
df.sample(5)

(421, 59)


,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
329,California,4/7/2017,NaN,NaN,NaN,6.50,4.0,NaN,NaN,21.3,23.8,0.96,3.0,3.0,4.0,4.0,4.0,4.0,3.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
336,Other,4/25/2017,NaN,NaN,NaN,10.00,4.2,925.0,0.600649,23.0,29.0,1.54,3.0,4.5,3.5,3.5,2.5,2.0,3.0,3.5,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
184,California,8/16/2016,NaN,NaN,NaN,5.29,3.5,NaN,NaN,22.5,20.5,0.75,2.5,4.5,2.0,3.0,2.5,1.5,3.0,2.0,4.5,NaN,NaN,x,x,NaN,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
217,Other,8/30/2016,NaN,NaN,NaN,7.50,5.0,NaN,NaN,NaN,NaN,NaN,3.5,2.0,3.5,3.5,3.0,4.0,2.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN,x,NaN,x,NaN,x,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
198,Surf & Turf,8/30/2016,NaN,NaN,NaN,8.25,3.5,NaN,NaN,20.0,22.0,0.77,3.6,3.2,3.2,3.5,3.6,3.5,NaN,3.8,3.9,NaN,NaN,x,x,NaN,NaN,NaN,NaN,NaN,NaN,x,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,x,NaN,NaN,False


In [8]:
df.isnull().sum()

Burrito             0
Date                0
Yelp              334
Google            334
Chips             395
Cost                7
Hunger              3
Mass (g)          399
Density (g/mL)    399
Length            138
Circum            140
Volume            140
Tortilla            0
Temp               20
Meat               14
Fillings            3
Meat:filling        9
Uniformity          2
Salsa              25
Synergy             2
Wrap                3
Unreliable        388
NonSD             414
Beef              242
Pico              263
Guac              267
Cheese            262
Fries             294
Sour cream        329
Pork              370
Chicken           400
Shrimp            400
Fish              415
Rice              385
Beans             386
Lettuce           410
Tomato            414
Bell peper        414
Carrots           420
Cabbage           413
Sauce             383
Salsa.1           414
Cilantro          406
Onion             404
Taquito           417
Pineapple 

#Train/ test/ val split

In [9]:
#creating a new feature with year 
df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)

year = []
[year.append(date.year) for date in df['Date']]
df['Year'] = year

df[['Year', 'Date']].sample(5)

,Year,Date
216,2016,2016-08-30
345,2017,2017-05-26
70,2016,2016-04-15
262,2016,2016-10-17
287,2016,2016-11-16


In [10]:
#doing the train test val split 
train = df[df['Year'] <= 2016]
val = df[df['Year'] == 2017]
test = df[df['Year'] > 2017]

train.shape, val.shape, test.shape

((298, 60), (85, 60), (38, 60))

#Baseline

In [11]:
#setting target variable
target = 'Great'
y_train = train[target]
y_val = val[target]
y_test = test[target]

In [12]:
#making the majority class for every guess
majority_class = y_train.mode()[0]
y_pred = [majority_class] * len(y_val)

In [13]:
#testing acuracy on val 
from sklearn.metrics import accuracy_score
accuracy_score(y_val, y_pred)

0.5529411764705883

In [14]:
#testing accuracy on testing data
y_pred = [majority_class] * len(y_test)
accuracy_score(y_test, y_pred)

0.42105263157894735

#Feature Cleaning (Stretch) 

In [15]:
#importing my cleaning tools
import category_encoders as ce 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [16]:
#creating the dataframes for the features (using them all for the first itteration except useless columns)
features = df.columns.drop(['Great', 'Year', 'Date', 'Google', 'Yelp', 'Queso','Mass (g)', 'Density (g/mL)'])
X_train = train[features]
X_val = val[features]
X_test = test[features]
print(X_train.shape, X_val.shape, X_test.shape)

(298, 52) (85, 52) (38, 52)


In [17]:
#encoding the categorical data to be used in the model
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.transform(X_val)
X_test_encoded = encoder.transform(X_test)
X_train_encoded.sample(5)

,Burrito_California,Burrito_Carnitas,Burrito_Asada,Burrito_Other,Burrito_Surf & Turf,Chips_nan,Chips_x,Chips_X,Cost,Hunger,Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Unreliable_nan,Unreliable_x,NonSD_nan,NonSD_x,NonSD_X,Beef_x,Beef_nan,Beef_X,Pico_x,Pico_nan,Pico_X,Guac_x,Guac_nan,Guac_X,Cheese_x,Cheese_nan,Cheese_X,Fries_x,...,Sauce_x,Sauce_X,Salsa.1_nan,Salsa.1_x,Salsa.1_X,Cilantro_nan,Cilantro_x,Cilantro_X,Onion_nan,Onion_x,Onion_X,Taquito_nan,Taquito_x,Taquito_X,Pineapple_nan,Pineapple_x,Pineapple_X,Ham_nan,Ham_x,Chile relleno_nan,Chile relleno_x,Nopales_nan,Nopales_x,Lobster_nan,Lobster_x,Egg_nan,Egg_x,Mushroom_nan,Mushroom_x,Bacon_nan,Bacon_x,Sushi_nan,Sushi_x,Avocado_nan,Avocado_x,Corn_nan,Corn_x,Corn_X,Zucchini_nan,Zucchini_x
201,0,0,0,0,1,1,0,0,8.25,4.0,NaN,NaN,NaN,4.0,3.0,3.0,3.0,3.0,2.0,3.0,3.0,4.0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,...,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,1,0,0,1,0
48,0,0,0,0,1,1,0,0,7.65,3.0,NaN,NaN,NaN,2.0,4.0,3.0,3.0,4.0,2.0,4.0,2.0,3.0,1,0,1,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,...,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0
134,0,0,0,1,0,1,0,0,NaN,4.3,NaN,NaN,NaN,3.8,4.0,NaN,4.7,NaN,5.0,NaN,4.7,5.0,0,1,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,...,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0
41,0,0,0,1,0,1,0,0,9.50,5.0,NaN,NaN,NaN,3.0,3.0,3.5,3.0,4.0,1.0,3.0,2.5,5.0,0,1,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,...,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0
283,0,0,1,0,0,1,0,0,6.75,3.0,18.0,21.5,0.66,4.5,4.7,4.5,3.5,4.0,4.5,4.5,3.5,5.0,1,0,1,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,...,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0


In [18]:
#imputing the missing values with the mean 
imputer = SimpleImputer()
X_train_imputed = imputer.fit_transform(X_train_encoded)
X_val_imputed = imputer.transform(X_val_encoded)
X_test_imputed = imputer.transform(X_test_encoded)
X_train_imputed.shape, X_val_imputed.shape, X_test_imputed.shape

((298, 118), (85, 118), (38, 118))

In [19]:
#putting all the data on the same scale
scaler = StandardScaler()
X_train_clean = scaler.fit_transform(X_train_imputed)
X_val_clean = scaler.transform(X_val_imputed)
X_test_clean = scaler.transform(X_test_imputed)

#Running Logistic Regression (Version 1)

In [20]:
#importing logistic regression and instantiating 
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [21]:
#fitting the model to the training data 
model.fit(X_train_clean, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [22]:
#running predictions on val data and testing accuracy 
y_pred = model.predict(X_val_clean)
accuracy_score(y_val, y_pred)

0.7647058823529411

In [31]:
y_pred = model.predict(X_test_clean)
accuracy_score(y_test, y_pred)

0.7631578947368421

In [23]:
X_train_encoded.shape

(298, 118)

In [29]:
#checking what features contributed the most to this result 
top_10 = pd.Series(model.coef_[0], X_train_encoded.columns).sort_values()[:10].index
bottom_10 = Top_40 = pd.Series(model.coef_[0], X_train_encoded.columns).sort_values()[108:118].index
top_20 = top_40.append(bottom_20)
top_20

Index(['Cost', 'Burrito_California', 'Pineapple_nan', 'Beans_nan', 'Tortilla',
       'Temp', 'Fillings', 'Meat:filling', 'Meat', 'Synergy', 'Cost',
       'Burrito_California', 'Pineapple_nan', 'Beans_nan', 'Tortilla', 'Temp',
       'Fillings', 'Meat:filling', 'Meat', 'Synergy'],
      dtype='object')

#Version 2 (only using top features)

In [36]:
#pulling just 20 features
features2 = ['Cost', 'Burrito_California', 'Pineapple_nan', 'Beans_nan', 'Tortilla',
       'Temp', 'Fillings', 'Meat:filling', 'Meat', 'Synergy', 'Cost',
       'Burrito_California', 'Pineapple_nan', 'Beans_nan', 'Tortilla', 'Temp',
       'Fillings', 'Meat:filling', 'Meat', 'Synergy']

X_train_2 = X_train_encoded[features2]
X_val_2 = X_val_encoded[features2]
X_test_2 = X_test_encoded[features2]

In [37]:
#imputing the new sets
X_train_imputed2 = imputer.fit_transform(X_train_2)
X_val_imputed2 = imputer.fit_transform(X_val_2)
X_test_imputed2 = imputer.fit_transform(X_test_2)

In [38]:
#scaling the new sets
X_train_scaled = scaler.fit_transform(X_train_imputed2)
X_val_scaled = scaler.transform(X_val_imputed2)
X_test_scaled = scaler.transform(X_test_imputed2)

In [39]:
#fitting the model to my new sets
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_val_scaled)

accuracy_score(y_val, y_pred)

0.8235294117647058

In [42]:
y_pred2 = model.predict(X_test_scaled)

accuracy_score(y_test, y_pred2)

0.7631578947368421

Didn't imporove my model but I shaved off 98 features!